<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Análisis de sentimiento
`Fecha de entrega: Noviembre 21, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
# cargar librerias 

# Estos paquetes los usaremos para analizar y visualizar los resultados
import pandas as pd
import tweepy
import numpy as np
import re
import regex
import emoji

# Libreias Plotly
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)
import seaborn as sns

from collections import Counter

import urllib.parse
import urllib.request
import time
import hmac
import hashlib
import random
import base64
import json

# Este es un modulo local de Python que contiene las credenciales de acceso para Twitter
import tweepy

from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

from flair.models import TextClassifier
from flair.data import Sentence

from wordcloud import WordCloud
from wordcloud import ImageColorGenerator

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')
import PIL.Image
from IPython.display import display

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning:

CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)



###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

### Repetimos el ejercio del taller anterior "taller10" parta obtener los datos de twiter 

In [2]:
# Leer llaves
keys = pd.read_csv("C:/Users/practicas/Documents/Claves Tuit.csv", header=None)
keys= dict(zip(keys[0],keys[1]))

In [3]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'],keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True) # se autoregula para recolectarinformacion

In [4]:
# Cargar archivo ciudades lat.-ln. bogota
ciudades = pd.read_csv("C:/Users/practicas/Documents/co_small.csv")
ciudades.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Bogotá,4.649178,-74.062827,Colombia,CO,Bogotá,primary,7772000.0,6333661.0
1,Medellín,6.251840,-75.563591,Colombia,CO,Antioquia,admin,3297000.0,1999979.0
2,Cali,3.437222,-76.522500,Colombia,CO,Valle del Cauca,admin,2254000.0,2178836.0
3,Barranquilla,10.963889,-74.796389,Colombia,CO,Atlántico,admin,1798000.0,1244491.0
4,Bucaramanga,7.125393,-73.119804,Colombia,CO,Santander,admin,1009000.0,571820.0


In [5]:
tweets = tweepy.Cursor(api.search,
                       q=['Selección Colombia futbol -RT'], # RT no incluir los retuis
                       lang='es', # solo tui en español
                           geocode=f'4.649178,-74.062827,45mi', #direccion geografica  bogota cambia a la que se quiera
                       tweet_mode='extended', # para que no recorte los texto del tuit
                       include_rts=False).items(100) # limitar el numero de twitwes

In [6]:
t = []
for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    
    t.append(temp)

In [7]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date']
data.head()

,tweet,date
0,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41
1,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47
2,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15
3,🧐 ¿Problemas internos en la Selección Colombia...,2020-11-19 02:37:58
4,Ancelotti sobre la posibilidad de dirigir a Co...,2020-11-19 02:34:55


In [8]:
#quitar enlaces y menciones
#quitar emojis

import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [9]:
#traducir tweet a ingles

translator = Translator()

data['ing'] = data['tweet'].apply(lambda valor: translator.translate(valor, src='es', dest='en'))
data.head()

,tweet,date,ing
0,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41,"Translated(src=es, dest=en, text=Which players..."
1,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47,"Translated(src=es, dest=en, text=Bluntly: Ance..."
2,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15,"Translated(src=es, dest=en, text=To the blows!..."
3,🧐 ¿Problemas internos en la Selección Colombia...,2020-11-19 02:37:58,"Translated(src=es, dest=en, text=🧐 Internal pr..."
4,Ancelotti sobre la posibilidad de dirigir a Co...,2020-11-19 02:34:55,"Translated(src=es, dest=en, text=Ancelotti on ..."


In [11]:
data['ingles'] = data['ing'].apply(lambda valor: valor.text)
data.head()

,tweet,date,ing,ingles
0,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41,"Translated(src=es, dest=en, text=Which players...",Which players fought in the dressing room of t...
1,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47,"Translated(src=es, dest=en, text=Bluntly: Ance...",Bluntly: Ancelotti answered if he would like t...
2,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15,"Translated(src=es, dest=en, text=To the blows!...",To the blows! There was a fight in the dressin...
3,🧐 ¿Problemas internos en la Selección Colombia...,2020-11-19 02:37:58,"Translated(src=es, dest=en, text=🧐 Internal pr...",🧐 Internal problems in the Colombian National ...
4,Ancelotti sobre la posibilidad de dirigir a Co...,2020-11-19 02:34:55,"Translated(src=es, dest=en, text=Ancelotti on ...",Ancelotti on the possibility of directing Colo...


## Librerias  para analisis de sentimientos vistas en clases
#### No es necesario limpiar la data

In [12]:
classifier = TextClassifier.load('sentiment')


def sent_flair(texto):
    sentence = Sentence(texto)
    classifier.predict(sentence)
    result = sentence.labels[0]
    label = result.value
    score = result.score
    if label == 'POSITIVE':
        return score
    if label == 'NEGATIVE':
        return -1 * score
    return score

 
data['sent_flair'] = data['tweet'].apply(lambda valor: sent_flair(valor))
data.head()

2020-11-18 22:36:44,401 loading file C:\Users\practicas\.flair\models\sentiment-en-mix-distillbert_3.1.pt


,tweet,date,ing,ingles,sent_flair
0,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41,"Translated(src=es, dest=en, text=Which players...",Which players fought in the dressing room of t...,-0.638301
1,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47,"Translated(src=es, dest=en, text=Bluntly: Ance...",Bluntly: Ancelotti answered if he would like t...,0.982045
2,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15,"Translated(src=es, dest=en, text=To the blows!...",To the blows! There was a fight in the dressin...,0.950778
3,🧐 ¿Problemas internos en la Selección Colombia...,2020-11-19 02:37:58,"Translated(src=es, dest=en, text=🧐 Internal pr...",🧐 Internal problems in the Colombian National ...,-0.822403
4,Ancelotti sobre la posibilidad de dirigir a Co...,2020-11-19 02:34:55,"Translated(src=es, dest=en, text=Ancelotti on ...",Ancelotti on the possibility of directing Colo...,0.988937


In [13]:
data.loc[4]['tweet']

'Ancelotti sobre la posibilidad de dirigir a Colombia y su opinión sobre Queiroz https://t.co/JCKAjtGIc0'

## Columna con el puntaje numérico del sentimiento

In [14]:
data[['tweet', 'sent_flair']].sort_values('sent_flair')


,tweet,sent_flair
35,@AmLuiissa No sirve. Ya tuvo su oportunidad co...,-0.999991
7,¿Camerino roto? Versiones sobre peleas y divis...,-0.999960
72,Opinión impopular: El salario del cuerpo técni...,-0.999810
57,"Colombia sin ""orgullo ni honor"" tras humillant...",-0.999481
59,¿De que otras formas necesitamos ser humilllad...,-0.999271
...,...,...
84,"Los procesos en el fútbol son muy complejos, C...",0.997595
56,Humillante 6-1: la peor derrota de la Selecció...,0.998198
73,"El señor Carlos Antonio Vélez sabe mucho, pero...",0.998571
80,Queiroz es el mejor técnico que ha tenido la S...,0.998609


### [10 pts] Punto 2: ¿Cuál es el tuit más positivo y más negativo?

In [15]:
#tuit Negativo
data.loc[99]['tweet']

'Vergonzosa presentación de la Selección Colombia en Quito https://t.co/olV3619Wfk'

In [16]:
#tuit Positivo
data.loc[68]['tweet']

'La crisis por la que pasa, el fútbol de la selección Colombia;  posiblemente nace en discrepancias, entre técnico y jugadores. Eso implica, un análisis profundo de la situación.'

### `[15 pts]` Punto 3: ¿Cuántos tuits positivos/negativos hay en sus datos?

In [17]:
data['resultado'] = " "
data.loc[data['sent_flair']>0, 'resultado'] =  'tweet Positivo'
data.loc[data['sent_flair']<0, 'resultado'] =  'tweet Negativo'
data.loc[data['sent_flair']==0, 'resultado'] = 'tweet Neutro'
data


,tweet,date,ing,ingles,sent_flair,resultado
0,¿Cuáles jugadores pelearon en el camerino de l...,2020-11-19 03:04:41,"Translated(src=es, dest=en, text=Which players...",Which players fought in the dressing room of t...,-0.638301,tweet Negativo
1,Sin rodeos: Ancelotti respondió si le gustaría...,2020-11-19 02:52:47,"Translated(src=es, dest=en, text=Bluntly: Ance...",Bluntly: Ancelotti answered if he would like t...,0.982045,tweet Positivo
2,¡A los golpes! Hubo pelea en el vestuario de l...,2020-11-19 02:51:15,"Translated(src=es, dest=en, text=To the blows!...",To the blows! There was a fight in the dressin...,0.950778,tweet Positivo
3,🧐 ¿Problemas internos en la Selección Colombia...,2020-11-19 02:37:58,"Translated(src=es, dest=en, text=🧐 Internal pr...",🧐 Internal problems in the Colombian National ...,-0.822403,tweet Negativo
4,Ancelotti sobre la posibilidad de dirigir a Co...,2020-11-19 02:34:55,"Translated(src=es, dest=en, text=Ancelotti on ...",Ancelotti on the possibility of directing Colo...,0.988937,tweet Positivo
...,...,...,...,...,...,...
95,Colombia fue humillada en Quito: cayó goleada ...,2020-11-18 00:03:55,"Translated(src=es, dest=en, text=Colombia was ...",Colombia was humiliated in Quito: they fell 6-...,0.986257,tweet Positivo
96,Juan Guillermo Cuadrado se refirió a la golead...,2020-11-18 00:00:46,"Translated(src=es, dest=en, text=Juan Guillerm...",Juan Guillermo Cuadrado referred to the win ag...,0.952445,tweet Positivo
97,Queiroz le pide tolerancia a los hinchas y se ...,2020-11-17 23:58:45,"Translated(src=es, dest=en, text=Queiroz asks ...",Queiroz asks the fans for tolerance and blames...,-0.561417,tweet Negativo
98,@ElDelaBudweiser @DeportesEE @elespectador (Si...,2020-11-17 23:56:53,"Translated(src=es, dest=en, text=@ElDelaBudwei...",@ElDelaBudweiser @DeportesEE @elespectador (Wi...,0.993289,tweet Positivo


In [18]:
# Conteo por tipo de sentimiento 
data = pd.DataFrame(data.resultado.value_counts()).reset_index()
data.columns = ['sent_flair', 'resultado']
data

,sent_flair,resultado
0,tweet Positivo,67
1,tweet Negativo,33


###  `[BONUS 5pts]` : Diagrama de barras de la pregunta 3


In [19]:
%matplotlib inline
init_notebook_mode(connected=True)

trace = go.Bar(x=data.sent_flair.values,
               y=data.resultado.values)

layout = go.Layout(title='Tweets x sentimientos')


In [20]:
fig = go.Figure(data = [trace], layout=layout)
iplot(fig)


###  `[BONUS 7pts]` : Nubes de palabras por sentimiento